In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv("full_dataset.csv")
df.head()

,Unnamed: 0,index,id,Name,Aritist,tag,Song_txt_loc,Song_lyrics
0,0,0,TRAAAED128E0783FAB,It's About Time,Jamie Cullum,relax,./lyrics_new/jamiecullum_itsabouttime.txt,Walking down to the water's edgeWhere I have ...
1,1,3,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,energetic,./lyrics_new/dillingerescapeplan_settingfireto...,First off Let me say you look so tired... Res...
2,2,7,TRAADVO128E07999E9,Oh God,Jamie Cullum,relax,./lyrics_new/jamiecullum_ohgod.txt,I know it's been a while since I have talked ...
3,3,8,TRAAFGQ128F427D884,One Last Time,The Kooks,sad,./lyrics_new/kooks_onelasttime.txt,Can I hold you one last time To fight the fee...
4,4,13,TRAAGPJ128F428CD1B,I Never Came,Queens Of The Stone Age,sad,./lyrics_new/queensofthestoneage_inevercame.txt,"When you say it's dead &amp; gone Yes, I know..."


In [3]:
df['tag'].value_counts()

love         5458
party        1630
sad          1304
emotional    1274
happy        1123
grunge       1034
memory        904
religious     892
dark          765
relax         531
political     511
energetic     506
christmas     474
funny         470
freedom       286
halloween     121
rain           81
Name: tag, dtype: int64

In [4]:
my_tags = ['christmans', 'dark', 'emotional', 'relax', 'freedom', 'funny', 'grunge', 'halloween',
           'happy', 'love', 'memory', 'party', 'political', 'rain', 'energetic', 'religious', 'sad']

In [13]:
print('No of words:', df['Song_lyrics'].apply(lambda x: len(x.split(' '))).sum())

No of words: 3852628


In [7]:
df['Song_lyrics'][2]

" I know it's been a while since I have talked to youBut maybe you're the one who makes the winds blowWe're looking at the stars without explanationWe contemplate as kings and simple men on trialOur little world's fragile\nOh God can you tell us when it's going to stopMaybe it's not just down to youOh God can we win back what we have lostSo who's the last resort... Oh God\nTumbling towards unclear destinationsDo they wash away the blame,The wind and the searing rainsAs our powers interchange\nOh God can you tell us when it's going to stopMaybe it's not just down to youOh God can we win back what we have lost  Oh God can you tell us when it's going to stopMaybe it's not just down to youOh God can we win back what we have lostSo who's the last resort"

In [8]:
df = df.replace('\n',' ', regex=True)
df['Song_lyrics'][2]

" I know it's been a while since I have talked to youBut maybe you're the one who makes the winds blowWe're looking at the stars without explanationWe contemplate as kings and simple men on trialOur little world's fragile Oh God can you tell us when it's going to stopMaybe it's not just down to youOh God can we win back what we have lostSo who's the last resort... Oh God Tumbling towards unclear destinationsDo they wash away the blame,The wind and the searing rainsAs our powers interchange Oh God can you tell us when it's going to stopMaybe it's not just down to youOh God can we win back what we have lost  Oh God can you tell us when it's going to stopMaybe it's not just down to youOh God can we win back what we have lostSo who's the last resort"

##    Cleaning lyrics

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 

df['lyrics_without_stopwords'] = df['Song_lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /home/jishnu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import string
df['lyrics_clean'] = df['lyrics_without_stopwords'].apply(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
df['lyrics_clean'] = df['lyrics_clean'].str.lower()

In [11]:
df1 = df[['lyrics_clean', 'tag']]
df1

,lyrics_clean,tag
0,walking waters edgewhere i beforeif i find lov...,relax
1,first let say look tired rest head shut eyes e...,energetic
2,i know since i talked youbut maybe one makes w...,relax
3,can i hold one last time to fight feeling grow...,sad
4,when say dead amp gone yes i know wrong cut am...,sad
...,...,...
17359,this shall pass oh always comes backand knocki...,religious
17360,he lord he reigns high he lord spoke darkness ...,religious
17361,exception occurred list index range,religious
17362,heres plani think i got madelife easywhen sitt...,religious


In [14]:
print('No of words:', df1['lyrics_clean'].apply(lambda x: len(x.split(' '))).sum())

No of words: 2354865


## ML models

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

X = df1.lyrics_clean
y = df1.tag
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [18]:
# Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [19]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.31362763915547026
              precision    recall  f1-score   support

  christmans       0.00      0.00      0.00       154
        dark       0.00      0.00      0.00       224
   emotional       0.00      0.00      0.00       388
       relax       0.00      0.00      0.00       161
     freedom       0.00      0.00      0.00        89
       funny       0.00      0.00      0.00       136
      grunge       0.00      0.00      0.00       303
   halloween       0.00      0.00      0.00        32
       happy       0.00      0.00      0.00       323
        love       0.31      1.00      0.48      1632
      memory       0.00      0.00      0.00       257
       party       1.00      0.00      0.01       519
   political       0.00      0.00      0.00       157
        rain       0.00      0.00      0.00        22
   energetic       0.00      0.00      0.00       156
   religious       0.00      0.00      0.00       256
         sad       0.00      0.00      0.00       40

/home/jishnu/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
# Linear SVM

from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [21]:
%%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.4295585412667946
              precision    recall  f1-score   support

  christmans       0.67      0.81      0.73       154
        dark       0.40      0.31      0.35       224
   emotional       0.37      0.20      0.26       388
       relax       0.15      0.04      0.06       161
     freedom       0.48      0.13      0.21        89
       funny       0.48      0.24      0.32       136
      grunge       0.42      0.26      0.32       303
   halloween       0.80      0.38      0.51        32
       happy       0.13      0.04      0.07       323
        love       0.45      0.82      0.59      1632
      memory       0.19      0.18      0.18       257
       party       0.50      0.38      0.43       519
   political       0.48      0.38      0.43       157
        rain       0.56      0.23      0.32        22
   energetic       0.10      0.02      0.03       156
   religious       0.43      0.52      0.47       256
         sad       0.18      0.06      0.09       401

In [22]:
# Logstic Regression 

from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

/home/jishnu/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jishnu/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [23]:
%%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.43550863723608446
              precision    recall  f1-score   support

  christmans       0.87      0.72      0.79       154
        dark       0.45      0.38      0.41       224
   emotional       0.35      0.28      0.31       388
       relax       0.09      0.04      0.06       161
     freedom       0.47      0.21      0.29        89
       funny       0.35      0.38      0.36       136
      grunge       0.37      0.28      0.32       303
   halloween       0.78      0.44      0.56        32
       happy       0.13      0.10      0.11       323
        love       0.52      0.69      0.59      1632
      memory       0.38      0.35      0.36       257
       party       0.52      0.53      0.53       519
   political       0.53      0.34      0.42       157
        rain       0.64      0.41      0.50        22
   energetic       0.04      0.03      0.03       156
   religious       0.51      0.49      0.50       256
         sad       0.17      0.18      0.17       40

In [28]:
# Random Forest
%%time

from sklearn.ensemble import RandomForestClassifier

rfc = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=500 ,random_state=0)),
               ])
rfc.fit(X_train, y_train)

CPU times: user 6min 54s, sys: 2.06 s, total: 6min 56s
Wall time: 6min 56s


In [29]:
%%time

y_pred = rfc.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.46775431861804223
              precision    recall  f1-score   support

  christmans       0.97      0.68      0.80       154
        dark       0.87      0.29      0.44       224
   emotional       0.93      0.16      0.28       388
       relax       0.67      0.02      0.05       161
     freedom       0.89      0.19      0.31        89
       funny       0.66      0.27      0.39       136
      grunge       0.97      0.19      0.32       303
   halloween       1.00      0.44      0.61        32
       happy       0.86      0.02      0.04       323
        love       0.38      0.99      0.54      1632
      memory       0.97      0.33      0.49       257
       party       0.90      0.46      0.61       519
   political       0.95      0.22      0.36       157
        rain       1.00      0.27      0.43        22
   energetic       0.25      0.01      0.01       156
   religious       0.85      0.32      0.47       256
         sad       0.88      0.02      0.03       40

In [47]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, y_pred)
accuracies = matrix.diagonal()/matrix.sum(axis=1)
for i in range(16):
    print(my_tags[i], ' : ', accuracies[i])

christmans  :  0.6753246753246753
dark  :  0.29464285714285715
emotional  :  0.16494845360824742
relax  :  0.024844720496894408
freedom  :  0.19101123595505617
funny  :  0.27205882352941174
grunge  :  0.19141914191419143
halloween  :  0.4375
happy  :  0.018575851393188854
love  :  0.9865196078431373
memory  :  0.33073929961089493
party  :  0.464354527938343
political  :  0.2229299363057325
rain  :  0.2727272727272727
energetic  :  0.00641025641025641
religious  :  0.3203125


In [48]:
ctr = 0
for i in range(len(y_pred)):
    if(y_pred[i]=='love'):
        ctr+=1

print(ctr)

4288


In [36]:
len(y_pred)

5210

## Word Embeddings

In [60]:
# create list of lists

lyrics = df1['lyrics_clean'].values.tolist()
sentences = [line.split() for line in lyrics]

In [61]:
# Pre trained word embeddings

from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True, limit=200000)
wv.init_sims(replace=True)

In [62]:
len(wv.vocab)

200000

In [63]:
def sentence_vectorizer(sentence):
    words = sentence.split()
    mean = np.mean([wv[word] for word in words if word in wv.vocab] or [np.zeros(300,)], axis=0)
    return mean

In [64]:
train_data = [sentence_vectorizer(sentence) for sentence in X] 

In [65]:
train_data[0].shape

(300,)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.3, random_state = 42)

In [68]:
%%time

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

/home/jishnu/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jishnu/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CPU times: user 39.9 s, sys: 31.4 ms, total: 39.9 s
Wall time: 39.9 s


In [69]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.3802303262955854
              precision    recall  f1-score   support

  christmans       0.69      0.69      0.69       154
        dark       0.29      0.20      0.23       224
   emotional       0.23      0.09      0.13       388
       relax       0.06      0.01      0.02       161
     freedom       0.15      0.06      0.08        89
       funny       0.28      0.18      0.22       136
      grunge       0.26      0.15      0.19       303
   halloween       0.15      0.16      0.15        32
       happy       0.10      0.02      0.04       323
        love       0.41      0.83      0.55      1632
      memory       0.22      0.04      0.07       257
       party       0.40      0.32      0.36       519
   political       0.39      0.27      0.32       157
        rain       0.08      0.18      0.11        22
   energetic       0.10      0.01      0.02       156
   religious       0.46      0.41      0.43       256
         sad       0.14      0.04      0.06       401

In [85]:
# Word2Vec embeddings 

import gensim
from gensim.models import Word2Vec

model = gensim.models.Word2Vec(sentences, min_count=5, size=50, window=5)

In [90]:
model.wv.most_similar('alone')[:5]

[('leave', 0.6968371272087097),
 ('lonely', 0.683151364326477),
 ('youleave', 0.6151683926582336),
 ('afraid', 0.6015790700912476),
 ('wondering', 0.6000429391860962)]

In [93]:
def sentence_vectorizer2(sentence):
    words = sentence.split()
    mean = np.mean([model[word] for word in words if word in model.wv.vocab] or [np.zeros(100,)], axis=0)
    return mean

In [94]:
train_data = [sentence_vectorizer2(sentence) for sentence in X] 

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
train_data[0].shape

(50,)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.3, random_state = 42)

In [97]:
X_train[0].shape

(50,)

In [98]:
%%time

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

/home/jishnu/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jishnu/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CPU times: user 6.04 s, sys: 15.7 ms, total: 6.06 s
Wall time: 6.06 s


In [99]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.36238003838771593
              precision    recall  f1-score   support

  christmans       0.77      0.62      0.69       154
        dark       0.22      0.08      0.11       224
   emotional       0.09      0.01      0.01       388
       relax       0.00      0.00      0.00       161
     freedom       0.00      0.00      0.00        89
       funny       0.21      0.04      0.06       136
      grunge       0.23      0.07      0.11       303
   halloween       0.00      0.00      0.00        32
       happy       0.07      0.00      0.01       323
        love       0.36      0.93      0.52      1632
      memory       0.00      0.00      0.00       257
       party       0.38      0.23      0.29       519
   political       0.23      0.11      0.15       157
        rain       0.50      0.05      0.08        22
   energetic       0.00      0.00      0.00       156
   religious       0.50      0.33      0.40       256
         sad       0.17      0.01      0.02       40

/home/jishnu/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [101]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=500 ,random_state=0)
rfc = rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [102]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.47715930902111325
              precision    recall  f1-score   support

  christmans       0.84      0.62      0.72       154
        dark       0.62      0.38      0.47       224
   emotional       0.92      0.17      0.28       388
       relax       0.67      0.02      0.05       161
     freedom       0.94      0.19      0.32        89
       funny       0.95      0.26      0.40       136
      grunge       0.84      0.20      0.32       303
   halloween       1.00      0.44      0.61        32
       happy       0.53      0.02      0.05       323
        love       0.40      0.97      0.56      1632
      memory       0.93      0.33      0.49       257
       party       0.67      0.55      0.60       519
   political       0.69      0.31      0.42       157
        rain       1.00      0.27      0.43        22
   energetic       0.33      0.01      0.01       156
   religious       0.75      0.33      0.46       256
         sad       0.57      0.02      0.04       40

In [116]:
matrix = confusion_matrix(y_test, y_pred)
accuracies = matrix.diagonal()/matrix.sum(axis=1)
for i in range(17):
    print(my_tags[i], ' : ', accuracies[i])

christmans  :  0.6233766233766234
dark  :  0.38392857142857145
emotional  :  0.16752577319587628
relax  :  0.024844720496894408
freedom  :  0.19101123595505617
funny  :  0.25735294117647056
grunge  :  0.20132013201320131
halloween  :  0.4375
happy  :  0.02476780185758514
love  :  0.9681372549019608
memory  :  0.33073929961089493
party  :  0.5529865125240848
political  :  0.3057324840764331
rain  :  0.2727272727272727
energetic  :  0.00641025641025641
religious  :  0.33203125
sad  :  0.0199501246882793


In [103]:
ctr = 0
for i in range(len(y_pred)):
    if(y_pred[i]=='love'):
        ctr+=1

print(ctr)

3994


In [106]:
model.wv.similarity('joy', 'happy')

0.327984

In [115]:
model.wv.most_similar('happy')[:5]

[('lucky', 0.6671848893165588),
 ('valentines', 0.581880509853363),
 ('lovely', 0.5433056950569153),
 ('someday', 0.5305999517440796),
 ('suspected', 0.5217271447181702)]

In [ ]:
len(model.wv.vocab)